### Prepare Notebook
Install dependencies, import packages, downloading dataset

In [ ]:
%%capture
!pip install openai langchain_openai langchain langchain_community
!pip install opendatasets

In [ ]:
from langchain import OpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
# from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import Document

import os
import json
import pandas

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ayeshacamran/mimic-clinical-notes/data")

In [ ]:
df = pandas.read_csv('/content/mimic-clinical-notes/NOTEEVENTS.csv/NOTEEVENTS.csv')
df.head()

<ipython-input-4-e7189007ebc9>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv('/content/mimic-clinical-notes/NOTEEVENTS.csv/NOTEEVENTS.csv')


,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


### Data Preprocessing


In [ ]:
df_long = df[df["TEXT"].str.len() > 25000]
anamnes = df_long.iloc[1]["TEXT"]
anamnes

"Admission Date:  [**2198-4-22**]              Discharge Date:   [**2198-5-4**]\n\nDate of Birth:  [**2160-7-23**]             Sex:   F\n\nService: MEDICINE\n\nAllergies:\nBactrim / Vioxx / Penicillins / CellCept / Ceftriaxone /\nFerrlecit / Sulfa (Sulfonamide Antibiotics)\n\nAttending:[**Last Name (NamePattern4) 290**]\nChief Complaint:\nHEMOPTYSIS\n\nMajor Surgical or Invasive Procedure:\nLEFT BRONCHIAL ARTERY EMBOLIZATION UNDER FLUOROSCOPY\nRIGHT INTERNAL JUGULAR LINE PLACEMENT, REPOSITIONING, AND\nREMOVAL\nIVC FILTER PLACEMENT\nINTUBATION AND MECHANICAL VENTILATION\n\nHistory of Present Illness:\n37F with history of lupus, lupus nephritis with ESRD on\nperitoneal dialysis on transplant list, hx of PE/Antiphopholipid\nantibody on coumadin, mitral regurg, presents with 4-6 month\nhistory of cough, worse in the morning, one week of trace blood,\nnow producing bright red blood over last couple days. Patient\nstates that the amount of blood she has been coughing has been\nincreasing and

In [ ]:
urinanalysis = """Physical Examination:

Color: Dark Yellow
Clarity: Cloudy
Chemical Examination:

pH: 5.0 (Normal range: 4.5-8.0)
Specific Gravity: 1.030 (Normal range: 1.005-1.030)
Glucose: Positive (+1) (Normal: Negative)
Ketones: Positive (+2) (Normal: Negative)
Protein: Positive (+1) (Normal: Negative)
Bilirubin: Positive (+1) (Normal: Negative)
Urobilinogen: Elevated (2.0 mg/dL) (Normal: 0.2 mg/dL)
Nitrite: Positive (Normal: Negative)
Leukocyte Esterase: Positive (+2) (Normal: Negative)
Blood: Positive (+2) (Normal: Negative)
Microscopic Examination:

Red Blood Cells (RBCs): 10-20 per HPF (High Power Field) (Normal: 0-2 per HPF)
White Blood Cells (WBCs): 20-50 per HPF (Normal: 0-5 per HPF)
Epithelial Cells: Moderate
Casts: Hyaline casts: 5-10 per LPF (Low Power Field) (Normal: None)
Crystals: Calcium oxalate crystals: Moderate
Bacteria: Many
Yeast: Few
Comments:

The urinalysis results indicate several abnormalities including glucose, ketones, protein, bilirubin, and blood in the urine. Elevated urobilinogen and presence of nitrites suggest possible liver dysfunction and urinary tract infection, respectively. The high number of WBCs and RBCs, along with the presence of bacteria and yeast, indicate a significant infection. Follow-up with a healthcare provider is recommended for further evaluation and management."""

In [ ]:
bloodanalysis = """
White Blood Cells (WBC): 12.5 x 10^9/L (Normal range: 4.0-11.0 x 10^9/L)
Red Blood Cells (RBC): 4.2 x 10^12/L (Normal range: 4.7-6.1 x 10^12/L for males)
Hemoglobin (Hgb): 11.0 g/dL (Normal range: 13.8-17.2 g/dL for males)
Hematocrit (Hct): 33% (Normal range: 40.7-50.3% for males)
Mean Corpuscular Volume (MCV): 78 fL (Normal range: 80-100 fL)
Mean Corpuscular Hemoglobin (MCH): 26 pg (Normal range: 27-31 pg)
Mean Corpuscular Hemoglobin Concentration (MCHC): 32 g/dL (Normal range: 32-36 g/dL)
Platelets: 250 x 10^9/L (Normal range: 150-450 x 10^9/L)
Differential Count:

Neutrophils: 75% (Normal range: 40-60%)
Lymphocytes: 15% (Normal range: 20-40%)
Monocytes: 6% (Normal range: 2-8%)
Eosinophils: 2% (Normal range: 1-4%)
Basophils: 2% (Normal range: 0.5-1%)
Basic Metabolic Panel (BMP):

Sodium (Na): 138 mmol/L (Normal range: 135-145 mmol/L)
Potassium (K): 4.5 mmol/L (Normal range: 3.5-5.0 mmol/L)
Chloride (Cl): 100 mmol/L (Normal range: 98-107 mmol/L)
Bicarbonate (HCO3): 24 mmol/L (Normal range: 22-28 mmol/L)
Blood Urea Nitrogen (BUN): 25 mg/dL (Normal range: 7-20 mg/dL)
Creatinine: 1.8 mg/dL (Normal range: 0.6-1.2 mg/dL)
Glucose: 140 mg/dL (Normal range: 70-99 mg/dL)
Liver Function Tests (LFTs):

Alanine Aminotransferase (ALT): 55 U/L (Normal range: 7-56 U/L)
Aspartate Aminotransferase (AST): 60 U/L (Normal range: 10-40 U/L)
Alkaline Phosphatase (ALP): 130 U/L (Normal range: 44-147 U/L)
Total Bilirubin: 2.1 mg/dL (Normal range: 0.1-1.2 mg/dL)
Comments:

The blood analysis reveals elevated white blood cells, indicating a possible infection or inflammatory response. Low red blood cells, hemoglobin, and hematocrit suggest anemia. The presence of microcytic red blood cells (low MCV) could indicate iron deficiency anemia. The elevated BUN and creatinine levels point to possible kidney dysfunction. Elevated glucose indicates hyperglycemia, which may require further investigation for diabetes. Abnormal liver function tests suggest liver inflammation or damage. Follow-up with a healthcare provider is recommended for further evaluation and management.
"""

### Prompt

In [ ]:
summary_prompt_template = """
You are a medical expert assistant. Given the following medical data, provide a concise short summary(without details in alalysis) highlighting the key abnormal findings:

Urinalysis:
{urinanalysis}

Blood Analysis:
{bloodanalysis}

Medical History:
{anamnes}

Summary:
"""

FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:
'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be sumamry of medical record
Action Input: the input to the action in python dictionary format
Observation: the result of the action
Final Answer: the final answer to the original input question
'''
"""

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.1, max_tokens=2048)

summary_prompt = PromptTemplate(
    input_variables=["urinanalysis", "bloodanalysis", "anamnes"],
    template=summary_prompt_template
)
llm_chain = LLMChain(llm=llm, prompt=summary_prompt)


def run_llm_chain(input_data):
    input_data = eval(input_data)
    return llm_chain.run(input_data)

summarize_tool = Tool.from_function(
    func=run_llm_chain,
    name="Summarizer",
    description="Summarizes a medical record"
)
tools = [summarize_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={
        'format_instructions': FORMAT_INSTRUCTIONS,
    },
    # handle_parsing_errors=True
)

input_data = {
    "urinanalysis": urinanalysis,
    "bloodanalysis": bloodanalysis,
    "anamnes": anamnes
}
summary_input = {"input": input_data}

summary = agent.run(summary_input)
print(summary)



> Entering new AgentExecutor chain...
To answer the question effectively, I need to summarize the provided medical records. 

Action: Summarizer(input_data)
Action Input: {
    'urinanalysis': 'Physical Examination:\n\nColor: Dark Yellow\nClarity: Cloudy\nChemical Examination:\n\npH: 5.0 (Normal range: 4.5-8.0)\nSpecific Gravity: 1.030 (Normal range: 1.005-1.030)\nGlucose: Positive (+1) (Normal: Negative)\nKetones: Positive (+2) (Normal: Negative)\nProtein: Positive (+1) (Normal: Negative)\nBilirubin: Positive (+1) (Normal: Negative)\nUrobilinogen: Elevated (2.0 mg/dL) (Normal: 0.2 mg/dL)\nNitrite: Positive (Normal: Negative)\nLeukocyte Esterase: Positive (+2) (Normal: Negative)\nBlood: Positive (+2) (Normal: Negative)\nMicroscopic Examination:\n\nRed Blood Cells (RBCs): 10-20 per HPF (High Power Field) (Normal: 0-2 per HPF)\nWhite Blood Cells (WBCs): 20-50 per HPF (Normal: 0-5 per HPF)\nEpithelial Cells: Moderate\nCasts: Hyaline casts: 5-10 per LPF (Low Power Field) (Normal: None)\n

In [ ]:
print(summary)

The medical data reveals several key abnormal findings:

1. **Urinalysis**:
   - Dark yellow, cloudy urine.
   - Positive for glucose, ketones, protein, bilirubin, nitrites, leukocyte esterase, and blood.
   - Elevated urobilinogen.
   - High RBCs (10-20 per HPF) and WBCs (20-50 per HPF).
   - Presence of bacteria, yeast, hyaline casts, and calcium oxalate crystals.

2. **Blood Analysis**:
   - Elevated WBC count (12.5 x 10^9/L).
   - Low RBC count (4.2 x 10^12/L), hemoglobin (11.0 g/dL), and hematocrit (33%).
   - Low MCV (78 fL) and MCH (26 pg).
   - Elevated BUN (25 mg/dL) and creatinine (1.8 mg/dL).
   - Elevated glucose (140 mg/dL).
   - Abnormal liver function tests: elevated ALT (55 U/L), AST (60 U/L), and total bilirubin (2.1 mg/dL).

3. **Radiologic Studies**:
   - Chest CT: Cavitary lesions in the left lung apex and moderate-sized left pleural effusion, consistent with an infectious process/tuberculosis.
   - Head CT: No intracranial hemorrhage or mass effect, but old infarct

### Refine Summarization

In [ ]:
full_medical_record = f"""
Urinalysis:
{urinanalysis}

Blood Analysis:
{bloodanalysis}

Medical History:
{anamnes}
"""

In [ ]:
def split_docs(text, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_text(text)
    return docs

docs = split_docs(full_medical_record)

In [ ]:
prompt_template = """Write a concise summary of the following medical record extracting the key information:

{text}

CONCISE SUMMARY:"""

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)

In [ ]:
prompt = PromptTemplate(template=prompt_template,
                        input_variables=["text"])

refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

chain = load_summarize_chain(llm,
                             chain_type="refine",
                             return_intermediate_steps=True,
                             question_prompt=prompt,
                             refine_prompt=refine_prompt)

docs = [Document(page_content=content) for content in docs]
output_summary = chain({"input_documents": docs}, return_only_outputs=True)

In [ ]:
output_summary['output_text']

"The urinalysis reveals a dark yellow, cloudy urine sample with several abnormalities. Chemical examination shows acidic pH (5.0), high specific gravity (1.030), and positive results for glucose, ketones, protein, bilirubin, nitrite, leukocyte esterase, and blood. Urobilinogen is elevated at 2.0 mg/dL. Microscopic examination indicates a high presence of RBCs (10-20 per HPF), WBCs (20-50 per HPF), moderate epithelial cells, hyaline casts (5-10 per LPF), moderate calcium oxalate crystals, many bacteria, and few yeast cells. The results suggest possible liver dysfunction and a significant urinary tract infection. Follow-up with a healthcare provider is recommended for further evaluation and management.\n\nBlood analysis shows elevated WBC count (12.5 x 10^9/L), indicating a possible infection. RBC count (4.2 x 10^12/L), hemoglobin (11.0 g/dL), and hematocrit (33%) are all below normal ranges, suggesting anemia. MCV (78 fL) and MCH (26 pg) are also below normal, indicating microcytic anem

In [ ]:
summary_prompt_template = """
You are a medical expert assistant. Given the following medical data, provide a concise short summary(without details in alalysis) highlighting the key abnormal findings:

Urinalysis:
{urinanalysis}

Blood Analysis:
{bloodanalysis}

Medical History:
{anamnes}

Summary:
"""

FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:
'''
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action in python dictionary format: \open_curly_bracket "urinanalysis": "urinanalysis info", "bloodanalysis": "bloodanalysis info", "anamnes": "anamnes info" \close_curly_bracket \eos
Observation: the result of the action
Final Answer: the final answer to the original input question
'''
"""

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

summary_prompt = PromptTemplate(
    input_variables=["urinanalysis", "bloodanalysis", "anamnes"],
    template=summary_prompt_template
)

llm_chain = LLMChain(
    llm=llm,
    prompt=summary_prompt
)


def run_llm_chain(input_data):
    print(len(input_data))
    input_data = eval(input_data)
    return llm_chain.run(input_data)

summarize_tool = Tool.from_function(
    func=run_llm_chain,
    name="Summarizer",
    description="Summarizes a medical record"
)
tools = [summarize_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={
        'format_instructions': FORMAT_INSTRUCTIONS,
    },
    # handle_parsing_errors=True
)

input_data = {
    "urinanalysis": urinanalysis,
    "bloodanalysis": bloodanalysis,
    "anamnes": anamnes2
}

summary_input = {"input": input_data}

summary = agent.run(summary_input)
# summary = run_llm_chain(input_data)
print(summary)



> Entering new AgentExecutor chain...
Do I need to use a tool? Yes
Action: Summarizer
Action Input: {"urinanalysis": "Physical Examination:\n\nColor: Dark Yellow\nClarity: Cloudy\nChemical Examination:\n\npH: 5.0 (Normal range: 4.5-8.0)\nSpecific Gravity: 1.030 (Normal range: 1.005-1.030)\nGlucose: Positive (+1) (Normal: Negative)\nKetones: Positive (+2) (Normal: Negative)\nProtein: Positive (+1) (Normal: Negative)\nBilirubin: Positive (+1) (Normal: Negative)\nUrobilinogen: Elevated (2.0 mg/dL) (Normal: 0.2 mg/dL)\nNitrite: Positive (Normal: Negative)\nLeukocyte Esterase: Positive (+2) (Normal: Negative)\nBlood: Positive (+2) (Normal: Negative)\nMicroscopic Examination:\n\nRed Blood Cells (RBCs): 10-20 per HPF (High Power Field) (Normal: 0-2 per HPF)\nWhite Blood Cells (WBCs): 20-50 per HPF (Normal: 0-5 per HPF)\nEpithelial Cells: Moderate\nCasts: Hyaline casts: 5-10 per LPF (Low Power Field) (Normal: None)\nCrystals: Calcium oxalate crystals: Moderate\nBacteria: Many\nYeast: Few\nCo